In [15]:
import pandas as pd
import os
import sys
sys.path.append('../src')
import cleansing as cl

In [16]:
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

import seaborn as sns
sns.set_context('poster')
sns.set(rc={'figure.figsize': (16., 9.)})
sns.set_style('whitegrid')
import plotly.express as px
import plotly.graph_objects as go

# Causas meses

In [87]:
causas_meses = pd.read_excel('../data/suicidio/diferenes_causas_.xlsx')
#ine acceso datos forenses cambian en 2013... solo datos de 2014 en adelante

In [88]:
causas_meses.sample(5)

,Sexo,Causa de muerte,Periodo,Total
473,Hombres,046-049 V.Trastornos mentales y del comportam...,1981M07,53
270,Hombres,046-049 V.Trastornos mentales y del comportam...,1998M06,292
3633,Mujeres,048 Trastornos mentales debidos al uso de dro...,2005M03,2
3395,Mujeres,047 Trastornos mentales debidos al uso de alc...,1984M01,3
1560,Hombres,049 Otros trastornos mentales y del comportam...,2013M12,28


In [89]:
causas_meses["year"] = causas_meses.Periodo.str[0:4].str.strip().astype('int64')
causas_meses["month"] = causas_meses.Periodo.str[-2:].str.strip().astype('int64')

In [90]:
causas_meses.columns = causas_meses.columns.str.replace(" ",'_').str.strip().str.lower()

In [91]:
#causas_meses = causas_meses[causas_meses['year']>=2014]

In [92]:
causas_meses.sample(3)

,sexo,causa_de_muerte,periodo,total,year,month
2254,Hombres,098 Suicidio y lesiones autoinfligidas,1997M02,194,1997,2
1285,Hombres,048 Trastornos mentales debidos al uso de dro...,1995M11,7,1995,11
4127,Mujeres,049 Otros trastornos mentales y del comportam...,2005M01,21,2005,1


In [93]:
causas_meses.causa_de_muerte.unique() #drop '046-049  V.Trastornos mentales y del comportamiento' porque engloba todo

array(['046-049  V.Trastornos mentales y del comportamiento',
       '047  Trastornos mentales debidos al uso de alcohol',
       '048  Trastornos mentales debidos al uso de drogas (drogodependencia, toxicoman�a)',
       '049  Otros trastornos mentales y del comportamiento',
       '098  Suicidio y lesiones autoinfligidas'], dtype=object)

In [94]:
dropv = causas_meses[causas_meses['causa_de_muerte'] == '046-049  V.Trastornos mentales y del comportamiento' ].index

In [95]:
causas_meses.drop(dropv, inplace = True)

In [115]:
mes_ = causas_meses.groupby(['year','month','sexo']).agg({'total':'sum'}).reset_index()

In [116]:
mes_.sample(3)

,year,month,sexo,total
563,2003,6,Mujeres,99
71,1982,12,Mujeres,52
825,2014,5,Mujeres,95


In [117]:
fig_sui = px.box(mes_, x="month", y="total", color = "sexo",title='suicidos por mes/género').update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(1,13)),
        ticktext = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre',
                    'Noviembre','Diciembre']
    )
)
fig_sui.show()


In [118]:
fig_sui.write_html('../output/fig_sui.html')

# Provincias causa sexo

In [66]:
provincia_causa = pd.read_excel('../data/suicidio/provincia_causa_anos.xlsx')

In [67]:
provincia_causa.sample(3) #046-049  V.Trastornos mentales y del comportamiento

,Provincias,Causa de muerte,Sexo,Periodo,Total
352,33 Asturias,046-049 V.Trastornos mentales y del comportam...,Hombres,1996,174
2549,30 Murcia,046-049 V.Trastornos mentales y del comportam...,Hombres,2013,109
2546,30 Murcia,046-049 V.Trastornos mentales y del comportam...,Hombres,2016,151


In [68]:
provincia_causa = provincia_causa[provincia_causa['Periodo']>1999]

In [69]:
provincia_causa.columns = provincia_causa.columns.str.replace(" ",'_').str.strip().str.lower()

In [70]:
provincia_causa.drop('causa_de_muerte',axis=1,inplace = True) #solo hay una causa

In [71]:
provincia_causa[['cod_pro', 'provincias']] = provincia_causa.provincias.str.split(' ',1,expand=True)

In [72]:
#hacemos el mapeo con función creada para la correspondencia con geojson
provincia_causa.provincias = provincia_causa.cod_pro.map(cl.provinciasjson())

In [73]:
provincia_causa.sample(3)

,provincias,sexo,periodo,total,cod_pro
4078,Zaragoza,Mujeres,2001,228,50
877,Burgos,Mujeres,2004,67,09
623,Baleares,Mujeres,2012,244,07


In [74]:
provincia_causa.provincias.fillna('Extranjero',inplace = True)

In [75]:
provincia_causa.cod_pro = provincia_causa.cod_pro.str.replace('Extranjero','53').astype('int64')

In [76]:
provincia_causa.sample(3)

,provincias,sexo,periodo,total,cod_pro
463,Ávila,Mujeres,2008,41,5
2466,Málaga,Hombres,2014,143,29
1112,Cantabria,Mujeres,2015,352,39


In [77]:
provincial = provincia_causa.groupby(['periodo','sexo']).agg({'total':'sum'}).reset_index()

In [82]:
fig_muer = px.line(provincial, x="periodo", y="total",color = 'sexo',title = 'Muertes por trastornos mentales y del comportamiento')
fig_muer.show()

In [84]:
fig_muer.write_html('../output/fig_muer.html')

# suicidio provincia year

In [28]:
suicidio_provincia_year = pd.read_excel('../data/suicidio/defunciones_causa_sexo_prov.xlsx')

In [29]:
suicidio_provincia_year.sample(3)

,Provincias,Causa de muerte,Sexo,Periodo,Total
1780,17 Girona,047 Trastornos mentales debidos al uso de alc...,Mujeres,2011,1
3206,36 Pontevedra,048 Trastornos mentales debidos al uso de dro...,Mujeres,2015,1
2640,29 Málaga,047 Trastornos mentales debidos al uso de alc...,Hombres,2020,9


In [30]:
suicidio_provincia_year.columns = ['provincia','causa','sexo','year','total']

In [31]:
suicidio_provincia_year.year = suicidio_provincia_year.year.astype('int64')

In [32]:
suicidio_provincia_year = suicidio_provincia_year[suicidio_provincia_year['year']>2013]

In [33]:
suicidio_provincia_year[['cod_pr','provincia']] = suicidio_provincia_year.provincia.str.split(' ',1,expand=True)

In [34]:
suicidio_provincia_year.sample(3)

,provincia,causa,sexo,year,total,cod_pr
727,Barcelona,048 Trastornos mentales debidos al uso de dro...,Hombres,2019,2,08
3724,Soria,048 Trastornos mentales debidos al uso de dro...,Hombres,2014,0,42
2216,Jaén,047 Trastornos mentales debidos al uso de alc...,Mujeres,2015,1,23


In [35]:
suicidio_provincia_year['provincia'] = suicidio_provincia_year.cod_pr.map(cl.provinciasjson())

In [36]:
suicidio_provincia_year.isna().sum()

provincia    56
causa         0
sexo          0
year          0
total         0
cod_pr        0
dtype: int64

In [37]:
suicidio_provincia_year[suicidio_provincia_year['provincia'].isna()]
suicidio_provincia_year.provincia.fillna('Extranjero',inplace = True)

In [38]:
suicidio_provincia_year.total=suicidio_provincia_year.total.replace('..','0').astype('int64')

In [39]:
suicidio_provincia_year.sample(3)

,provincia,causa,sexo,year,total,cod_pr
1797,Girona,048 Trastornos mentales debidos al uso de dro...,Mujeres,2016,0,17
3949,Teruel,098 Suicidio y lesiones autoinfligidas,Mujeres,2020,2,44
677,Baleares,049 Otros trastornos mentales y del comportam...,Mujeres,2014,10,07


# suicidio edad sexo_year

In [40]:
suicidio_edad_sexo = pd.read_excel('../data/suicidio/suicidios_lesiones.xlsx')

In [41]:
suicidio_edad_sexo.sample(3)

,Causa de muerte,Sexo,Edad,Periodo,Total
107,098 Suicidio y lesiones autoinfligidas,Hombres,De 5 a 9 a�os,1995,0
747,098 Suicidio y lesiones autoinfligidas,Hombres,De 85 a 89 a�os,2011,92
214,098 Suicidio y lesiones autoinfligidas,Hombres,De 20 a 24 a�os,2011,60


In [42]:
suicidio_edad_sexo.columns = suicidio_edad_sexo.columns.str.replace(' ','_').str.lower()

In [43]:
suicidio_edad_sexo.causa_de_muerte.unique() #'098  Suicidio y lesiones autoinfligidas'

array(['098  Suicidio y lesiones autoinfligidas'], dtype=object)

In [44]:
suicidio_edad_sexo.drop('causa_de_muerte',axis=1,inplace = True)

In [45]:
#menmos de 5 no hay, quitamos
menores = ['Menos de 1 a�o','De 1 a 4 a�os']
menores_drop = suicidio_edad_sexo[suicidio_edad_sexo['edad'].isin(menores)].index

In [46]:
suicidio_edad_sexo.drop(menores_drop,inplace = True)

In [47]:
suicidio_edad_sexo.edad = suicidio_edad_sexo.edad.str.strip()

In [48]:
suicidio_edad_sexo.edad = suicidio_edad_sexo.edad.str.extract(r'((\d+)\D*$)')[1].astype('int64')

In [49]:
suicidio_edad_sexo = suicidio_edad_sexo[suicidio_edad_sexo['periodo']>2013]

In [50]:
suicidio_edad_sexo.sample(5)

,sexo,edad,periodo,total
985,Mujeres,14,2019,3
1559,Mujeres,84,2019,51
1069,Mujeres,24,2017,16
703,Hombres,84,2014,184
1642,Mujeres,94,2018,17


# tasas 2020

## tasas  'Trastornos mentales y del comportamiento'

In [85]:
tasas = pd.read_excel('../data/suicidio/tasas2020.xlsx')

In [86]:
tasas.sample(3)

,Comunidades y ciudades autónomas,Causa de muerte (capítulos de la 10Ş revisión de la CIE),Sexo,Tasa/coeficiente de variación,Total
8,Canarias,Trastornos mentales y del comportamiento,Hombres,Tasa estandarizada (tanto por cien mil),29.65
11,Cantabria,Trastornos mentales y del comportamiento,Mujeres,Tasa estandarizada (tanto por cien mil),51.38
17,Cataluña,Trastornos mentales y del comportamiento,Mujeres,Tasa estandarizada (tanto por cien mil),41.75


In [52]:
tasas.columns = ['ccaa','causa','sexo','tasa','total']

In [53]:
tasas.causa.unique() #'Trastornos mentales y del comportamiento']

array(['Trastornos mentales y del comportamiento'], dtype=object)

In [54]:
tasas.tasa.unique() #'Tasa estandarizada (tanto por cien mil)'

array(['Tasa estandarizada (tanto por cien mil)'], dtype=object)

In [55]:
tasas.drop(['causa','tasa'],axis=1,inplace = True)

In [56]:
tasas.sample(3)

,ccaa,sexo,total
1,Andalucía,Mujeres,32.18
31,País Vasco,Mujeres,55.73
11,Cantabria,Mujeres,51.38


In [57]:
fig = px.box(tasas, x="ccaa", y="total",color = 'sexo')
fig.show()

## tasas suicidio

In [123]:
suicidios = pd.read_excel('../data/suicidio/tasassuicidios2020.xlsx') 
#Tasas de suicidios por edad y sexo por 100.000 habitantes de cada grupo.

In [125]:
suicidios.head(20)

,Edad,Sexo,Total
0,Menores de 15 años,Hombres,0.199
1,Menores de 15 años,Mujeres,0.212
2,De 15 a 19 años,Hombres,2.478
3,De 15 a 19 años,Mujeres,1.453
4,De 20 a 24 años,Hombres,6.656
5,De 20 a 24 años,Mujeres,1.642
6,De 25 a 29 años,Hombres,8.940
7,De 25 a 29 años,Mujeres,2.953
8,De 30 a 34 años,Hombres,10.068
9,De 30 a 34 años,Mujeres,2.460


In [ ]:
#Tasas de suicidios por edad y sexo por 100.000 habitantes de cada grupo.

In [126]:
fig = px.bar(suicidios, x="Edad", y="Total",color = 'Sexo',
            title = 'Tasas de suicidios por edad y sexo por 100.000 habitantes de cada grupo.'
            )
fig.show()